<a href="https://colab.research.google.com/github/amrashraf15/RegularExpression-To-DFA/blob/main/REtoDFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regular Expression to Minimized DFA



In [20]:
!pip install graphviz


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import json
from graphviz import Digraph
from itertools import count

## State Class

In [22]:
class State:
    c = count(0)
    def __init__(self, name=None):
        if name is None:
            self.name = f"S{next(State.c)}"
        else:
            self.name = name
        self.transitions = {}
        self.is_EndState = False
        self.is_StartState = False

## NFA Class

In [ ]:
class NFA:
    def __init__(self, start=None, end=None):
        self.start = start
        self.end = end
        self.startState = start
        self.states = {}
        self.acceptingStates = set()
        self.alphabet = set()
    
    def from_json(self, json_data):
        data = json.loads(json_data)
        start_state_name = data["startingState"]
        
        for state_name, state_info in data.items():
            if state_name == "startingState":
                continue
            
            state = State(state_name)
            state.is_EndState = state_info.get("isTerminatingState", False)
            
            if state_name == start_state_name:
                state.is_StartState = True
                self.startState = state
                self.start = state
            
            if state.is_EndState:
                self.acceptingStates.add(state_name)
                if self.end is None:
                    self.end = state
            
            self.states[state_name] = state
        
        for state_name, state_info in data.items():
            if state_name == "startingState":
                continue
            
            current_state = self.states[state_name]
            
            for symbol, transitions in state_info.items():
                if symbol == "isTerminatingState":
                    continue
                
                if not isinstance(transitions, list):
                    transitions = [transitions]
                
                if symbol not in current_state.transitions:
                    current_state.transitions[symbol] = set()
                
                for next_state in transitions:
                    current_state.transitions[symbol].add(next_state)
                
                if symbol != "ε" and symbol != "epsilon":
                    self.alphabet.add(symbol)

## PART 1: Regular Expression to NFA

In [ ]:
def create_tokens(regex):
    tokens = []
    i = 0

    while i < len(regex):
        char = regex[i]

        if char == '[':
            j = i + 1
            while j < len(regex) and regex[j] != ']':
                j += 1
            if j == len(regex):
                raise ValueError("Unclosed [ in regex")
            tokens.append(regex[i:j+1])
            i = j + 1
            continue

        if char == '.':
            tokens.append('.')
            i += 1
            continue

        if char in "*+?|()":
            tokens.append(char)
            i += 1
            continue

        if char.isalnum():
            tokens.append(char)
            i += 1
            continue

        raise ValueError(f"Invalid character: {char}")

    return tokens

### Regex to Postfix Conversion (Shunting Yard Algorithm)

In [ ]:
precedence = {'*': 3, '+': 3, '?': 3, '&': 2, '|': 1}

def addConcat(tokens):
    result = []

    for i in range(len(tokens)):
        result.append(tokens[i])

        if i + 1 < len(tokens):
            left = tokens[i]
            right = tokens[i+1]

            if (left not in "|(" and left != '&') and \
               (right not in "|)*+?"):
                result.append('&')

    return result

def toPostfix(regex):
    tokens = create_tokens(regex)
    tokens = addConcat(tokens)

    output = []
    stack = []

    for token in tokens:

        if token not in precedence and token not in "()|":
            output.append(token)

        elif token == '(':
            stack.append(token)

        elif token == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            stack.pop()

        elif token in precedence:
            while stack and stack[-1] != '(' and precedence.get(stack[-1], 0) >= precedence[token]:
                output.append(stack.pop())
            stack.append(token)

    while stack:
        output.append(stack.pop())

    return output

### Thompson's Construction Algorithm

In [ ]:
def thompson(postfix):
    stack = []

    for token in postfix:
        if token not in precedence and token not in ('&', '|', '*', '+', '?'):
            s0 = State()
            s1 = State()
            s0.transitions[token] = {s1}
            stack.append(NFA(s0, s1))

        elif token == '&':
            nfa2 = stack.pop()
            nfa1 = stack.pop()
            nfa1.end.transitions['ε'] = {nfa2.start}
            stack.append(NFA(nfa1.start, nfa2.end))

        elif token == '|':
            nfa2 = stack.pop()
            nfa1 = stack.pop()
            s0 = State()
            s1 = State()
            s0.transitions['ε'] = {nfa1.start, nfa2.start}
            nfa1.end.transitions['ε'] = {s1}
            nfa2.end.transitions['ε'] = {s1}
            stack.append(NFA(s0, s1))

        elif token == '*':
            nfa1 = stack.pop()
            s0 = State()
            s1 = State()
            s0.transitions['ε'] = {nfa1.start, s1}
            nfa1.end.transitions['ε'] = {nfa1.start, s1}
            stack.append(NFA(s0, s1))

        elif token == '+':
            nfa1 = stack.pop()
            s0 = State()
            s1 = State()
            s0.transitions['ε'] = {nfa1.start}
            nfa1.end.transitions['ε'] = {nfa1.start, s1}
            stack.append(NFA(s0, s1))

        elif token == '?':
            nfa1 = stack.pop()
            s0 = State()
            s1 = State()
            s0.transitions['ε'] = {nfa1.start, s1}
            nfa1.end.transitions['ε'] = {s1}
            stack.append(NFA(s0, s1))

    nfa = stack.pop()
    nfa.end.is_EndState = True
    return nfa

### NFA to JSON Export

In [ ]:
def nfaTojson(nfa, filename):
    states = {}
    visited = set()

    def dfs(state):
        if state.name in visited:
            return
        visited.add(state.name)
        transitions = {"isTerminatingState": state.is_EndState}
        for symbol, next_states in state.transitions.items():
            transitions[symbol] = [s.name for s in next_states]
            for s in next_states:
                dfs(s)
        states[state.name] = transitions

    dfs(nfa.start)
    data = {"startingState": nfa.start.name}
    data.update(states)

    if filename:
        with open(filename, "w") as f:
            json.dump(data, f, indent=2)
    return data

### NFA Visualization

In [ ]:
def draw_nfa(nfa, filename="nfa_graph"):
    dot = Digraph(format="png")
    dot.attr(rankdir="LR")

    visited = set()

    def dfs(state):
        if state.name in visited:
            return
        visited.add(state.name)


        if state.is_EndState:
            dot.node(state.name, shape="doublecircle")
        else:
            dot.node(state.name, shape="circle")

        for symbol, next_states in state.transitions.items():
            for s in next_states:
                label = "ε" if symbol == "ε" else symbol
                dot.edge(state.name, s.name, label=label)
                dfs(s)


    dot.node("start", shape="plaintext", label="")
    dot.edge("start", nfa.start.name)

    dfs(nfa.start)

    dot.render(filename, cleanup=True)
    print(f"Graph saved as {filename}.png")

## PART 2: NFA to DFA Conversion

### DFA State Class

In [28]:
class DFAState:
    def __init__(self, nfa_states):
        self.nfa_states = frozenset(nfa_states)
        self.name = self._generate_name()
        self.transitions = {}
        self.is_accepting = False
        self.is_start = False
    
    def _generate_name(self):
        if not self.nfa_states:
            return "{∅}"
        sorted_states = sorted(self.nfa_states)
        return "{" + ",".join(sorted_states) + "}"
    
    def __hash__(self):
        return hash(self.nfa_states)
    
    def __eq__(self, other):
        if not isinstance(other, DFAState):
            return False
        return self.nfa_states == other.nfa_states
    
    def __repr__(self):
        return f"DFAState({self.name})"

### DFA Class

In [ ]:
class DFA:
    def __init__(self):
        self.startState = None
        self.states = {}
        self.transitions = {} 
        self.acceptingStates = set()
    
    def get_reachable_states(self):
        if self.startState is None:
            return set()
        
        reachable = set()
        queue = [self.startState.name]
        visited = {self.startState.name}
        
        while queue:
            current_name = queue.pop(0)
            reachable.add(current_name)
            
            if current_name not in self.states:
                continue
            
            current_state = self.states[current_name]
            
            for symbol, next_state_name in current_state.transitions.items():
                if next_state_name not in visited:
                    visited.add(next_state_name)
                    queue.append(next_state_name)
        
        return reachable
    
    def from_json(self, json_data):
        data = json.loads(json_data)
        start_state_name = data["startingState"]
        
        for state_name, state_info in data.items():
            if state_name == "startingState":
                continue
            
            state = State(state_name)
            state.is_EndState = state_info["isTerminatingState"]
            
            if state_name == start_state_name:
                state.is_StartState = True
                self.startState = state
            
            if state.is_EndState:
                self.acceptingStates.add(state_name)
            
            self.states[state_name] = state
        
        for state_name, state_info in data.items():
            if state_name == "startingState":
                continue
            
            current_state = self.states[state_name]
            
            for symbol, next_state_name in state_info.items():
                if symbol != "isTerminatingState":
                    current_state.transitions[symbol] = next_state_name
    
    def Minimize(self):
        reachable = self.get_reachable_states()
        unreachable = set(self.states.keys()) - reachable
        
        for state_name in unreachable:
            del self.states[state_name]
            if state_name in self.acceptingStates:
                self.acceptingStates.remove(state_name)
        
        if not self.states:
            return
        
        reverse_edges = {}
        alphabet = set()
        
        for state_name, state in self.states.items():
            for symbol, target in state.transitions.items():
                alphabet.add(symbol)
                
                if symbol not in reverse_edges:
                    reverse_edges[symbol] = {}
                if target not in reverse_edges[symbol]:
                    reverse_edges[symbol][target] = set()
                
                reverse_edges[symbol][target].add(state_name)
        
        accepting = self.acceptingStates & set(self.states.keys())
        non_accepting = set(self.states.keys()) - accepting
        
        partitions = []
        if non_accepting:
            partitions.append(non_accepting)
        if accepting:
            partitions.append(accepting)
        
        if len(partitions) == 0:
            return
        
        queue = []
        if len(partitions) == 2:
            if len(accepting) <= len(non_accepting):
                queue.append(accepting)
            else:
                queue.append(non_accepting)
        else:
            queue.append(partitions[0])
        
        partition_set = set()
        for p in partitions:
            partition_set.add(frozenset(p))
        
        while queue:
            splitter = queue.pop(0)
            
            for symbol in alphabet:
                states_to_splitter = set()
                for target_state in splitter:
                    if symbol in reverse_edges and target_state in reverse_edges[symbol]:
                        states_to_splitter.update(reverse_edges[symbol][target_state])
                
                new_partitions = []
                for partition in partitions:
                    partition_frozen = frozenset(partition)
                    
                    goes_to_splitter = partition & states_to_splitter
                    not_goes_to_splitter = partition - states_to_splitter
                    
                    if goes_to_splitter and not_goes_to_splitter:
                        new_partitions.append(goes_to_splitter)
                        new_partitions.append(not_goes_to_splitter)
                        
                        partition_frozen = frozenset(partition)
                        
                        if len(goes_to_splitter) <= len(not_goes_to_splitter):
                            queue.append(goes_to_splitter)
                            if partition_frozen in [frozenset(q) for q in queue]:
                                queue.append(not_goes_to_splitter)
                        else:
                            queue.append(not_goes_to_splitter)
                            if partition_frozen in [frozenset(q) for q in queue]:
                                queue.append(goes_to_splitter)
                    else:
                        new_partitions.append(partition)
                
                partitions = new_partitions
        
        state_to_partition = {}
        partition_representatives = {}
        
        for idx, partition in enumerate(partitions):
            if not partition:
                continue
            
            sorted_partition = sorted(partition)
            
            if self.startState.name in partition:
                representative = self.startState.name
            else:
                representative = sorted_partition[0]
            
            partition_representatives[idx] = representative
            
            for state_name in partition:
                state_to_partition[state_name] = idx
        
        new_states = {}
        new_accepting = set()
        new_start = None
        
        for idx, partition in enumerate(partitions):
            if not partition:
                continue
            
            rep = partition_representatives[idx]
            new_state = State(rep)
            
            if partition & self.acceptingStates:
                new_state.is_EndState = True
                new_accepting.add(rep)
            
            if self.startState.name in partition:
                new_state.is_StartState = True
                new_start = new_state
            
            old_state = self.states[list(partition)[0]]
            for symbol, old_next in old_state.transitions.items():
                next_partition_idx = state_to_partition[old_next]
                next_rep = partition_representatives[next_partition_idx]
                new_state.transitions[symbol] = next_rep
            
            new_states[rep] = new_state
        
        self.states = new_states
        self.acceptingStates = new_accepting
        self.startState = new_start
    
    def to_json(self):
        result = {"startingState": self.startState.name}
        
        for state_name, state in sorted(self.states.items()):
            state_dict = {"isTerminatingState": state.is_EndState}
            for symbol, next_state in sorted(state.transitions.items()):
                state_dict[symbol] = next_state
            result[state_name] = state_dict
        
        return json.dumps(result, indent=2)
    
    def visualize(self, filename="dfa_graph"):
        dot = Digraph(comment='DFA')
        
        dot.attr(rankdir='LR')
        dot.attr('graph', 
                 bgcolor='white',
                 dpi='300',
                 fontname='Arial',
                 fontsize='12',
                 splines='true',
                 overlap='false',
                 nodesep='0.8',
                 ranksep='1.2')
        
        dot.attr('node',
                 fontname='Arial',
                 fontsize='14',
                 style='filled',
                 fontcolor='black')
        
        dot.attr('edge',
                 fontname='Arial',
                 fontsize='12',
                 fontcolor='black',
                 color='#2C3E50',
                 arrowsize='0.8')
        
        dot.node('__start__', '', shape='none', width='0', height='0')
        
        for state_name, state in self.states.items():
            if state_name == "DEAD_STATE":
                continue
            if state.is_EndState:
                dot.node(state_name, state_name,
                        shape='doublecircle',
                        fillcolor='#90EE90',
                        color='#228B22',
                        penwidth='2.5')
            else:
                dot.node(state_name, state_name,
                        shape='circle',
                        fillcolor='#87CEEB',
                        color='#1E90FF',
                        penwidth='2.5')
        
        dot.edge('__start__', self.startState.name,
                color='#FF6347',
                penwidth='2.5',
                label='START',
                fontcolor='#FF6347',
                fontsize='11',
                fontweight='bold')
        
        edge_labels = {}
        for state_name, state in self.states.items():
            if state_name == "DEAD_STATE":
                continue
            for symbol, next_state_name in state.transitions.items():
                if next_state_name == "DEAD_STATE":
                    continue
                edge_key = (state_name, next_state_name)
                if edge_key not in edge_labels:
                    edge_labels[edge_key] = []
                edge_labels[edge_key].append(symbol)
        
        for (source, destination), symbols in edge_labels.items():
            label = ', '.join(sorted(symbols))
            
            if source == destination:
                dot.edge(source, destination,
                        label=label,
                        color='#2C3E50',
                        penwidth='2.0',
                        fontsize='11')
            else:
                dot.edge(source, destination,
                        label=label,
                        penwidth='2.0',
                        fontsize='11')
        
        try:
            dot.render(filename, format='png', cleanup=True)
            print(f"✓ Graph saved as {filename}.png")
            with open(f'{filename}.dot', 'w') as f:
                f.write(dot.source)
            print(f"✓ DOT source saved as {filename}.dot")
            
        except Exception as e:
            print(f"✗ Error generating graph: {e}")
            print("\nNote: Graphviz requires both:")
            print("1. Python package: pip install graphviz")
            print("2. Graphviz executables: https://graphviz.org/download/")
            print("   After installation, add Graphviz bin folder to system PATH")
            print("\n💡 Tip: You can copy the DOT source and paste it at:")
            print("   https://dreampuf.github.io/GraphvizOnline/")
            
        return dot

### Epsilon Closure Algorithm

In [ ]:
def epsilon_closure(nfa, state_set):
    closure = set()
    visited = set()
    stack = list(state_set)
    
    while stack:
        current = stack.pop()
        
        if current in visited:
            continue
        
        visited.add(current)
        closure.add(current)
        
        if current not in nfa.states:
            continue
        
        state = nfa.states[current]
        
        epsilon_targets = set()
        if 'ε' in state.transitions:
            epsilon_targets.update(state.transitions['ε'])
        if 'epsilon' in state.transitions:
            epsilon_targets.update(state.transitions['epsilon'])
        
        for target in epsilon_targets:
            if target not in visited:
                stack.append(target)
    
    return closure

### Move Function (Transition on Symbol)

In [31]:
def move(nfa, state_set, symbol):
    result = set()
    
    for state_name in state_set:
        if state_name not in nfa.states:
            continue
        
        state = nfa.states[state_name]
        
        if symbol in state.transitions:
            result.update(state.transitions[symbol])
    
    return result

### Subset Construction Algorithm (NFA to DFA Conversion)

In [ ]:
def nfa_to_dfa(nfa):
    if nfa.startState is None:
        print("Error: NFA has no start state")
        return DFA()
    
    initial_nfa_set = epsilon_closure(nfa, {nfa.startState.name})
    initial_dfa_state = DFAState(initial_nfa_set)
    initial_dfa_state.is_start = True
    
    if initial_nfa_set & nfa.acceptingStates:
        initial_dfa_state.is_accepting = True
    
    queue = [initial_dfa_state]
    processed = {initial_dfa_state.nfa_states: initial_dfa_state}
    all_dfa_states = [initial_dfa_state]
    
    while queue:
        current_dfa_state = queue.pop(0)
        
        for symbol in sorted(nfa.alphabet):
            next_nfa_set = move(nfa, current_dfa_state.nfa_states, symbol)
            
            next_nfa_set = epsilon_closure(nfa, next_nfa_set)
            
            if not next_nfa_set:
                dead_key = frozenset()
                
                if dead_key not in processed:
                    dead_state = DFAState(set())
                    dead_state.name = "DEAD_STATE"
                    dead_state.is_accepting = False
                    processed[dead_key] = dead_state
                    all_dfa_states.append(dead_state)
                    
                    for sym in nfa.alphabet:
                        dead_state.transitions[sym] = dead_state.name
                
                current_dfa_state.transitions[symbol] = processed[dead_key].name
                continue
            
            next_frozen = frozenset(next_nfa_set)
            
            if next_frozen not in processed:
                new_dfa_state = DFAState(next_nfa_set)
                
                if next_nfa_set & nfa.acceptingStates:
                    new_dfa_state.is_accepting = True
                
                processed[next_frozen] = new_dfa_state
                all_dfa_states.append(new_dfa_state)
                queue.append(new_dfa_state)
            
            next_dfa_state = processed[next_frozen]
            current_dfa_state.transitions[symbol] = next_dfa_state.name
    
    dfa = DFA()
    dfa.startState = State(initial_dfa_state.name)
    dfa.startState.is_StartState = True
    dfa.startState.is_EndState = initial_dfa_state.is_accepting
    
    if initial_dfa_state.is_accepting:
        dfa.acceptingStates.add(initial_dfa_state.name)
    
    for dfa_state in all_dfa_states:
        if dfa_state.name not in dfa.states:
            state = State(dfa_state.name)
            state.is_EndState = dfa_state.is_accepting
            state.is_StartState = dfa_state.is_start
            state.transitions = dfa_state.transitions.copy()
            dfa.states[dfa_state.name] = state
            
            if dfa_state.is_accepting:
                dfa.acceptingStates.add(dfa_state.name)
    
    return dfa

## Utility Functions

In [33]:
def reset_state_counter():
    State.c = count(0)

## Complete Pipeline: Regex to Minimized DFA

In [ ]:
def regex_to_minimized_dfa(regex, save_json=True, visualize=True):
    reset_state_counter()
    
    print("="*70)
    print("STEP 1: Regex to Postfix")
    print("="*70)
    postfix = toPostfix(regex)
    print(f"Regex: {regex}")
    print(f"Postfix: {postfix}")
    
    print("\n" + "="*70)
    print("STEP 2: Postfix to NFA (Thompson's Construction)")
    print("="*70)
    nfa = thompson(postfix)
    print(f"NFA Start State: {nfa.start.name}")
    print(f"NFA End State: {nfa.end.name}")
    
    nfa_data = nfaTojson(nfa, "nfa.json" if save_json else None)
    
    if save_json:
        print("\nNFA JSON structure:")
        print(json.dumps(nfa_data, indent=2))
    
    if visualize:
        draw_nfa(nfa, "nfa_graph")
    
    print("\n" + "="*70)
    print("STEP 3: NFA to DFA (Subset Construction)")
    print("="*70)
    
    nfa_json_data = json.dumps(nfa_data)
    nfa_obj = NFA()
    nfa_obj.from_json(nfa_json_data)
    
    dfa = nfa_to_dfa(nfa_obj)
    
    dfa_states_display = [s for s in dfa.states.keys() if s != "DEAD_STATE"]
    dfa_accepting_display = {s for s in dfa.acceptingStates if s != "DEAD_STATE"}
    
    print(f"DFA States: {dfa_states_display}")
    print(f"DFA Start State: {dfa.startState.name}")
    print(f"DFA Accepting States: {dfa_accepting_display}")
    
    print("\nDFA Transitions:")
    for state_name in sorted(dfa.states.keys()):
        if state_name == "DEAD_STATE":
            continue
        state = dfa.states[state_name]
        for symbol in sorted(state.transitions.keys()):
            next_state = state.transitions[symbol]
    
    min_dfa_states_display = [s for s in dfa.states.keys() if s != "DEAD_STATE"]
    min_dfa_accepting_display = {s for s in dfa.acceptingStates if s != "DEAD_STATE"}
    
    print(f"Minimized DFA States: {min_dfa_states_display}")
    print(f"Minimized DFA Start State: {dfa.startState.name}")
    print(f"Minimized DFA Accepting States: {min_dfa_accepting_display}")
    
    print("\nMinimized DFA Transitions:")
    for state_name in sorted(dfa.states.keys()):
        if state_name == "DEAD_STATE":
            continue
        state = dfa.states[state_name]
        for symbol in sorted(state.transitions.keys()):
            next_state = state.transitions[symbol]
            if next_state == "DEAD_STATE":
                continue
            print(f"  {state_name} --{symbol}--> {next_state}")
    
    if save_json:
        json_output = dfa.to_json()
        with open("minimized_dfa.json", "w") as f:
            f.write(json_output)
        print("\nMinimized DFA JSON:")
        print(json_output)

    return nfa, dfa        print("="*70)    print("PIPELINE COMPLETE")    print("\n" + "="*70)            dfa.visualize("minimized_dfa")    if visualize:        
    if visualize:
        dfa.visualize("minimized_dfa")
    
    print("\n" + "="*70)
    print("PIPELINE COMPLETE")
    print("="*70)
    
    return nfa, dfa

## Example Usage

In [36]:
regex = "(ab)+c"
nfa, dfa = regex_to_minimized_dfa(regex)

STEP 1: Regex to Postfix
Regex: (ab)+c
Postfix: ab.+c.

STEP 2: Postfix to NFA (Thompson's Construction)
NFA Start State: S4
NFA End State: S7

NFA JSON structure:
{
  "startingState": "S4",
  "S7": {
    "isTerminatingState": true
  },
  "S6": {
    "isTerminatingState": false,
    "c": "S7"
  },
  "S5": {
    "isTerminatingState": false,
    "e": "S6"
  },
  "S3": {
    "isTerminatingState": false,
    "e": "S0"
  },
  "S2": {
    "isTerminatingState": false,
    "b": "S3"
  },
  "S1": {
    "isTerminatingState": false,
    "e": "S2"
  },
  "S0": {
    "isTerminatingState": false,
    "a": "S1"
  },
  "S4": {
    "isTerminatingState": false,
    "e": "S0"
  }
}


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH